In [ ]:
import librosa
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline 

In [ ]:
y, sr = librosa.load(librosa.util.example_audio_file())
D = np.abs(librosa.stft(y))
C = np.abs(librosa.cqt(y))

In [ ]:
librosa.display.specshow(librosa.amplitude_to_db(D, ref=np.max), y_axis='log', x_axis='time')
plt.title('Power spectrogram')
plt.colorbar(format='%+2.0f dB')
plt.tight_layout()

In [ ]:
librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max), y_axis='cqt_note', x_axis='time')
plt.title('Constant-Q power spectrum')
plt.colorbar(format='%+2.0f dB')
plt.tight_layout()